In [38]:
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling

In [39]:
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=3, n_obj=2, n_ieq_constr=0, xl=np.array([0, 0, 0]), xu=np.array([50, 50, 50]), vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        poblacion = np.round(x).astype(int)
        To, Fr, M = poblacion[:, 0], poblacion[:, 1], poblacion[:, 2]
        
        ganancia = -(4*To+8*Fr+12*M)
        produccion = ((abs(2*To+Fr+5*M-280) + abs(6*To+3*Fr+12*M-120) +
                    abs(2*To+8*Fr+4*M-670) + abs(2*To+6*Fr+6*M-250) +
                    abs(To-Fr) + abs(To-M) + abs(Fr-M)))
        out["F"] = np.column_stack([produccion, ganancia])
problem = MyProblem()

In [40]:
results = minimize(
            problem = problem,
            algorithm=NSGA2(pop_size=200),
            termination=('n_gen', 1000),
            seed=1,
            save_history=True
        )

In [41]:
resultados = results.F.T
fig = go.Figure(data=go.Scatter(x=resultados[0], y=resultados[1], mode='markers'))
fig.show()

In [55]:
print("Best solution found: %s" % results.X)

Best solution found: [[4.97026279e+01 4.99863728e+01 4.95962633e+01]
 [2.41172200e-01 4.17594637e+01 2.63550580e-03]
 [4.07177029e+01 4.97057551e+01 4.97404462e+01]
 [3.64511319e+01 4.96718028e+01 4.99851652e+01]
 [3.89525071e+01 4.97510788e+01 4.99877493e+01]
 [4.46323108e+01 4.97364959e+01 4.99372901e+01]
 [3.81786033e+01 4.96172052e+01 4.99058805e+01]
 [4.66921318e+01 4.96940491e+01 4.99252252e+01]
 [3.98817243e+01 4.97057551e+01 4.97404462e+01]
 [4.33759239e+01 4.95003567e+01 4.99360486e+01]
 [4.39277761e+01 4.98094300e+01 4.99388450e+01]
 [4.83951503e+01 4.96940491e+01 4.99252252e+01]
 [3.73462549e+01 4.97510788e+01 4.98889817e+01]
 [4.56986988e+01 4.96940491e+01 4.99252252e+01]
 [4.20812581e+01 4.99913094e+01 4.99513896e+01]
 [4.86483245e+01 4.99567550e+01 4.98524634e+01]
 [3.46280811e+01 4.99868836e+01 4.98524634e+01]
 [3.72546667e+01 4.96172052e+01 4.86640159e+01]
 [4.12734173e+01 4.98571027e+01 4.74611794e+01]
 [3.89525071e+01 4.96911179e+01 4.76852610e+01]
 [4.34378239e+01 4.